In [1]:
import spacy
import torch
from tqdm import tqdm
from src.model import Transformer
from torchtext import data, datasets
from torchtext import data, datasets
from src.loss import MultiGPULossCompute
from src.training import (
    LabelSmoothing, DataIterator, rebatch,
    batch_size_function, NoamOptimizer, train_step
)

In [2]:
spacy_source = spacy.load('de')
spacy_target = spacy.load('en')

In [3]:
def tokenize_source(text):
    return [tok.text for tok in spacy_source.tokenizer(text)]

def tokenize_target(text):
    return [tok.text for tok in spacy_target.tokenizer(text)]

In [4]:
source = data.Field(tokenize=tokenize_source, pad_token="<blank>")
target = data.Field(
    tokenize=tokenize_target, init_token="<s>",
    eos_token="</s>", pad_token="<blank>"
)

In [5]:
max_length = 100

train, val, test = datasets.IWSLT.splits(
    exts=('.de', '.en'), fields=(source, target), 
    filter_pred=lambda x: len(vars(x)['src']) \
    <= max_length and len(vars(x)['trg']) <= max_length
)

In [6]:
min_frequency = 2

source.build_vocab(train.src, min_freq=min_frequency)
target.build_vocab(train.trg, min_freq=min_frequency)

In [7]:
pad_idx = target.vocab.stoi["<blank>"]
model = Transformer(len(source.vocab), len(target.vocab), n=6)
model.cuda()

print(model)

/notebooks/transformer.pytorch/src/model.py:76: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(p)


EncoderDecoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attention): MultiHeadedAttention(
          (linear_layers): ModuleList(
            (0): Linear(in_features=512, out_features=512, bias=True)
            (1): Linear(in_features=512, out_features=512, bias=True)
            (2): Linear(in_features=512, out_features=512, bias=True)
            (3): Linear(in_features=512, out_features=512, bias=True)
          )
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): PositionWiseFeedForward(
          (w_1): Linear(in_features=512, out_features=2048, bias=True)
          (w_2): Linear(in_features=2048, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (sublayer): ModuleList(
          (0): ResidualConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): ResidualConnection(
            (norm): 

In [8]:
criterion = LabelSmoothing(
    size=len(target.vocab),
    padding_index=pad_idx, smoothing=0.1
)
criterion.cuda()

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


LabelSmoothing(
  (criterion): KLDivLoss()
)

In [10]:
train_iter = DataIterator(
    train, batch_size=1200, device=0,
    repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
    batch_size_fn=batch_size_function, train=True
)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [ ]:
valid_iter = DataIterator(
    val, batch_size=1200, device=0,
    repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
    batch_size_fn=batch_size_function, train=False
)

In [ ]:
model_parameters = torch.nn.DataParallel(model, device_ids=[0])

In [ ]:
model_optimizer = NoamOptimizer(
    model.source_embedding[0].d_model, 1, 2000,
    torch.optim.Adam(
        model.parameters(), lr=0,
        betas=(0.9, 0.98), eps=1e-9
    )
)

In [ ]:
def train(epochs):
    for epoch in range(epochs):
        print('Epochs:', (epochs + 1))
        model_parameters.train()
        train_step(
            (rebatch(pad_idx, b) for b in train_iter), 
            model_par, MultiGPULossCompute(
                model.generator, criterion,
                devices=[0], opt=model_optimizer
            )
        )
        model_par.eval()
        loss = train_step(
            (rebatch(pad_idx, b) for b in valid_iter), 
            model_par, MultiGPULossCompute(
                model.generator, criterion, 
                devices=devices, opt=None
            )
        )

In [11]:
def train(epochs):
    for epoch in range(epochs):
        print('Epochs:', (epochs + 1))
        model_parameters.train()
        train_step(
            (rebatch(pad_idx, b) for b in train_iter), 
            model_par, MultiGPULossCompute(
                model.generator, criterion,
                devices=[0], opt=model_optimizer
            )
        )
        model_par.eval()
        loss = train_step(
            (rebatch(pad_idx, b) for b in valid_iter), 
            model_par, MultiGPULossCompute(
                model.generator, criterion, 
                devices=devices, opt=None
            )
        )

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [12]:
model_parameters = torch.nn.DataParallel(model, device_ids=[0])

In [13]:
model_optimizer = NoamOptimizer(
    model.source_embedding[0].d_model, 1, 2000,
    torch.optim.Adam(
        model.parameters(), lr=0,
        betas=(0.9, 0.98), eps=1e-9
    )
)

In [ ]:
def train(epochs):
    for epoch in range(epochs):
        print('Epochs:', (epochs + 1))
        model_parameters.train()
        train_step(
            (rebatch(pad_idx, b) for b in train_iter), 
            model_parameters, MultiGPULossCompute(
                model.generator, criterion,
                devices=[0], opt=model_optimizer
            )
        )
        model_parameters.eval()
        loss = train_step(
            (rebatch(pad_idx, b) for b in valid_iter), 
            model_parameters, MultiGPULossCompute(
                model.generator, criterion, 
                devices=devices, opt=None
            )
        )